In [6]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [7]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test, edgeindex2adj
from models import GCN, GAT, LP
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')

In [8]:
# dataset = citeseer
# model = GCN(dataset.num_features, 24, dataset.num_classes)

# dataset = cora
# model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [9]:
from models import ALP, GCN
dataset = cora
data = dataset[0]

# take k random training nodes for each class
k = 3
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

print(data.train_mask.sum())

tensor(21)


In [13]:
# run for 5 seeds
av_acc = 0

for seed in range(5):
    torch.manual_seed(seed)
    model = GCN(dataset.num_features, 8, dataset.num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=50, min_lr=0.0001)

    best_val_acc = final_test_acc = 0
    for epoch in range(1, 250):
        loss = train(model, data, optimizer, scheduler=None, loss='smooth_label', alpha=20)
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch % 25 == 0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')
    av_acc += test_acc
print(f'Average Test Acc: {av_acc/5:.4f}')

Epoch: 025, Loss: 1.8628408908843994, Train: 0.3810, Val: 0.1580, Test: 0.2980
Epoch: 050, Loss: 1.802296757698059, Train: 0.3810, Val: 0.1160, Test: 0.2980
Epoch: 075, Loss: 1.7939027547836304, Train: 0.3810, Val: 0.0740, Test: 0.2980
Epoch: 100, Loss: 1.793192744255066, Train: 0.3810, Val: 0.1580, Test: 0.2980


KeyboardInterrupt: 

In [ ]:
# run for 5 seeds
av_acc = 0

for seed in range(5):
    torch.manual_seed(seed)
    model = GCN(dataset.num_features, 8, dataset.num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=50, min_lr=0.0001)

    best_val_acc = final_test_acc = 0
    for epoch in range(1, 150):
        loss = train(model, data, optimizer, scheduler=scheduler, loss='cross_entropy')
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch % 25 == 0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')
    av_acc += test_acc
print(f'Average Test Acc: {av_acc/5:.4f}')

Epoch: 025, Loss: 0.1498504877090454, Train: 1.0000, Val: 0.4900, Test: 0.5050
Epoch: 050, Loss: 0.00931228045374155, Train: 1.0000, Val: 0.5380, Test: 0.5520
Epoch: 075, Loss: 0.007171915378421545, Train: 1.0000, Val: 0.5620, Test: 0.5710
Epoch: 100, Loss: 0.007477668579667807, Train: 1.0000, Val: 0.5600, Test: 0.5710
Epoch: 125, Loss: 0.007068636827170849, Train: 1.0000, Val: 0.5620, Test: 0.5710
Best Val Acc: 0.5740 Test Acc: 0.5910
Epoch: 025, Loss: 0.0841929242014885, Train: 1.0000, Val: 0.4880, Test: 0.5240
Epoch: 050, Loss: 0.009557830169796944, Train: 1.0000, Val: 0.5300, Test: 0.5490
Epoch: 075, Loss: 0.007854974828660488, Train: 1.0000, Val: 0.5440, Test: 0.5590
Epoch: 100, Loss: 0.007982530631124973, Train: 1.0000, Val: 0.5500, Test: 0.5620
Epoch: 125, Loss: 0.007707900833338499, Train: 1.0000, Val: 0.5620, Test: 0.5790
Best Val Acc: 0.5820 Test Acc: 0.5890
Epoch: 025, Loss: 0.1472441405057907, Train: 1.0000, Val: 0.4940, Test: 0.4930
Epoch: 050, Loss: 0.009941385127604008, 

In [ ]:
# dataset = citeseer
# dataset = cora
dataset = pubmed

best_val_acc = final_test_acc = 0
best_l = best_a = 0

for l in [1, 2, 4, 8, 16, 32]:
    for a in [0.05, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.99, 1]:
        model = LP(num_layers=l, alpha=a)
        outs = model.train(dataset)
        train_acc, val_acc, tmp_test_acc = model.test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            best_l = l
            best_a = a
            
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}', f'Best l: {best_l}', f'Best a: {best_a}')

Best Val Acc: 0.7260 Test Acc: 0.7140 Best l: 16 Best a: 0.99


In [ ]:
accs = model.test()
accs

[0.8833333333333333, 0.712, 0.707]

In [ ]:
outs.shape

torch.Size([19717, 3])

In [ ]:
outs.argmax(dim=-1)

tensor([1, 1, 0,  ..., 2, 0, 2])

In [ ]:
dataset[0].y

tensor([1, 1, 0,  ..., 2, 0, 2])

In [ ]:
dataset.num_classes

3